In [32]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt

In [33]:
import pandas as pd
import numpy as np

TS = 48 # Time steps

df = pd.read_csv('AusGrid_preprocess.csv', header=[0,1], index_col=0)
df = df.set_index(pd.to_datetime(df.index))
df.columns = df.columns.set_levels(df.columns.levels[0].astype('int64'), level=0)
df.head()

Customer             1                    3                 4           \
categories            GG      GC      CL   GG      GC   CL   GG     GC   
2012-07-01 00:30:00  0.0  1710.0  2500.0  0.0  1930.0  0.0  0.0  168.0   
2012-07-01 01:00:00  0.0  1572.0  2500.0  0.0  1854.0  0.0  0.0  168.0   
2012-07-01 01:30:00  0.0  1208.0  2500.0  0.0  2718.0  0.0  0.0  164.0   
2012-07-01 02:00:00  0.0  1088.0  2526.0  0.0   120.0  0.0  0.0  168.0   
2012-07-01 02:30:00  0.0  1194.0   262.0  0.0   118.0  0.0  0.0  172.0   

Customer                     5    ...  297                 298         299  \
categories               CL   GG  ...   GG     GC      CL   GG     GC   GG   
2012-07-01 00:30:00     0.0  0.0  ...  0.0  282.0     0.0  0.0  314.0  0.0   
2012-07-01 01:00:00     0.0  0.0  ...  0.0  514.0     0.0  0.0  254.0  0.0   
2012-07-01 01:30:00  3860.0  0.0  ...  0.0  394.0     0.0  0.0  244.0  0.0   
2012-07-01 02:00:00  3956.0  0.0  ...  0.0  546.0  2660.0  0.0  240.0  0.0   
2012-07-01 02:30:00  1568.0  0.0  ...  0.0  412.0     0.0  0.0  212.0  0.0   

Customer                     300                 
categories               GC   GG     GC      CL  
2012-07-01 00:30:00  1082.0  0.0  438.0  5346.0  
2012-07-01 01:00:00   200.0  0.0  198.0  2492.0  
2012-07-01 01:30:00   180.0  0.0  268.0     0.0  
2012-07-01 02:00:00   188.0  0.0  200.0     0.0  
2012-07-01 02:30:00    92.0  0.0  246.0     0.0  

[5 rows x 725 columns]

In [34]:
customers = sorted(df.columns.levels[0])
max_values = []
for c in customers:
    max_values.append(df[c]['GG'].max())

normalize = max(max_values)
print(normalize)



8662.0


In [35]:
def gen_data(X, y, num_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - num_steps):        
        Xs.append(np.reshape(X[i:(i + num_steps)], (num_steps, 1)))
        ys.append(y[i + num_steps])
    return np.array(Xs), np.reshape(np.array(ys), (len(ys)))
    
customers = sorted(df.columns.levels[0])
x_list = []
y_list = []
for c in customers:
    tmp = df[c]['GG'].to_numpy()/normalize
    tmp_x, tmp_y = gen_data(tmp, tmp, TS)        
    x_list.append(tmp_x)
    y_list.append(tmp_y)
x_data = np.array(x_list)
y_data = np.array(y_list)
print(x_data.shape, y_data.shape)


(299, 17472, 48, 1) (299, 17472)


In [36]:
x_train, y_train = x_data[:200], y_data[:200]
print(x_train.shape, y_train.shape)
x_test, y_test = x_data[200:], y_data[200:]
print(x_test.shape, y_test.shape)

(200, 17472, 48, 1) (200, 17472)
(99, 17472, 48, 1) (99, 17472)


In [37]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# x_train = np.expand_dims(x_train, -1)  # 채널 차원 추가
# x_test = np.expand_dims(x_test, -1)  # 채널 차원 추가
# y_train = tf.keras.utils.to_categorical(y_train, 10)
# y_test = tf.keras.utils.to_categorical(y_test, 10)
x_train.shape

(200, 17472, 48, 1)

In [22]:
# 연합학습을 위한 데이터 분할
num_clients = 5
client_data_size = len(x_train) // num_clients
client_data = [(x_train[i * client_data_size: (i + 1) * client_data_size],
                y_train[i * client_data_size: (i + 1) * client_data_size])
               for i in range(num_clients)]
# len(client_data)
client_data[0][1].shape

(40, 17472, 1)

In [23]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

def create_model():
    model = Sequential([
        LSTM(256, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True),
        LSTM(16),
        Dense(1)
    ])
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [24]:
# 모델 생성
model = create_model()
model.summary()

/Users/jwheo/Projects/etc_project/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 17472, 256)     │       312,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 16)             │        17,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 329,809 (1.26 MB)

 Trainable params: 329,809 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# 클라이언트 모델 초기화
def initialize_client_models(num_clients, global_weights=None):
    client_models = []
    for _ in range(num_clients):
        model = create_model()
        if global_weights is not None:
            model.set_weights(global_weights)  # 모든 클라이언트 모델에 동일한 글로벌 가중치 설정
        client_models.append(model)
    return client_models

# FedAvg로 글로벌 모델 업데이트 함수
def federated_averaging(client_models, client_data):
    global_model = create_model()
    
    # 클라이언트 데이터 샘플 수를 기준으로 가중치 계산
    num_samples = [data[0].shape[0] for data in client_data]
    total_samples = sum(num_samples)
    client_weights = [samples / total_samples for samples in num_samples]
    
    # 각 클라이언트의 가중치를 평균화
    model_weights = [model.get_weights() for model in client_models]
    average_weights = []
    for weights_list in zip(*model_weights):
        average_weights.append(np.average(weights_list, axis=0, weights=client_weights))
    
    global_model.set_weights(average_weights)
    return global_model

# 각 클라이언트 모델 학습 함수
def train_client_model(client_data, model, epochs=1, batch_size=16):
    history = model.fit(client_data[0], client_data[1], epochs=epochs, batch_size=batch_size, verbose=1)
    return history

def evaluate_global(model, x_test, y_test):
    total_test_loss = 0.0
    total_test_acc = 0.0
    for (x_ev, y_ev) in zip(x_test, y_test):
        test_loss, test_acc = model.evaluate(x_ev, y_ev, verbose=0)
        total_test_loss += test_loss
        total_test_acc += test_acc

    return total_test_loss/(len(x_test)), total_test_acc/(len(y_test))
        
# 연합 학습 반복 함수
def federated_learning(num_rounds, num_clients, client_data):
    # 초기 글로벌 모델 설정
    global_model = create_model()
    global_model_accuracies = []
    
    for round in range(num_rounds):
        print(f"Round {round + 1}/{num_rounds}")
        
        # 클라이언트 모델 초기화
        client_models = initialize_client_models(num_clients, global_weights=global_model.get_weights())
        
        # 클라이언트 모델 학습
        for i in range(num_clients):
            print(f"Training client model {i + 1}...")
            train_client_model(client_data[i], client_models[i])
        '''
        # FedAvg로 글로벌 모델 생성
        global_model = federated_averaging(client_models, client_data)
        
        # 글로벌 모델 평가
        # test_loss, test_acc = global_model.evaluate(x_test[0], y_test[0], verbose=0)
        test_loss, test_acc = evaluate_global(global_model, x_test, y_test)
        global_model_accuracies.append(test_acc * 100)
        print(f'Global model test accuracy: {test_acc * 100:.2f}%')
        
        # 학습 상황 및 최종 결과 그래프
        # client_accuracies = [model.evaluate(x_test, y_test, verbose=0)[1] for model in client_models]
        client_accuracies = [evaluate_global(model, x_test, y_test)[1] for model in client_models]

        plt.figure(figsize=(10, 6))
        plt.plot(range(num_clients), client_accuracies, 'bo-', label='Client Models')
        plt.axhline(y=test_acc, color='r', linestyle='-', label='Global Model')
        plt.xlabel('Client')
        plt.ylabel('Accuracy')
        plt.title(f'Client Models vs Global Model Accuracy (Round {round + 1})')
        plt.legend()
        plt.show()
        # 특정 테스트 인덱스를 선택하여 예측 결과 시각화
        num_samples = x_test.shape[0]
        test_index = np.random.randint(num_samples)

        plt.figure(figsize=(12, 6))

        for i in range(num_clients):
            plt.subplot(2, num_clients, i + 1)
            plt.imshow(x_test[test_index].squeeze(), cmap='gray')
            client_prediction = np.argmax(client_models[i].predict(x_test[test_index:test_index+1]))
            true_label = np.argmax(y_test[test_index])
            plt.title(f"Client {i+1}\nPred: {client_prediction}\nTrue: {true_label}")
            plt.axis('off')

        plt.subplot(2, num_clients, num_clients + 1)
        plt.imshow(x_test[test_index].squeeze(), cmap='gray')
        global_prediction = np.argmax(global_model.predict(x_test[test_index:test_index+1]))
        true_label = np.argmax(y_test[test_index])
        plt.title(f"Global Model\nPred: {global_prediction}\nTrue: {true_label}")
        plt.axis('off')

        plt.show()
        '''

    return global_model, global_model_accuracies

In [26]:
print(x_test.shape, y_test.shape)

# 연합 학습 수행
num_rounds = 1
global_model, global_model_accuracies = federated_learning(num_rounds, num_clients, client_data)

(99, 17472, 48, 1) (99, 17472, 1)
Round 1/1
Training client model 1...
3/3 ━━━━━━━━━━━━━━━━━━━━ 357s 94s/step - loss: 0.0034
Training client model 2...
3/3 ━━━━━━━━━━━━━━━━━━━━ 199s 60s/step - loss: 0.0059
Training client model 3...
3/3 ━━━━━━━━━━━━━━━━━━━━ 219s 70s/step - loss: 0.0045
Training client model 4...
3/3 ━━━━━━━━━━━━━━━━━━━━ 230s 68s/step - loss: 0.0048
Training client model 5...
3/3 ━━━━━━━━━━━━━━━━━━━━ 266s 83s/step - loss: 0.0032


In [ ]:
print(y_test.shape)
# 최종 글로벌 모델 평가
# test_loss, test_acc = global_model.evaluate(x_test, y_test, verbose=0)
test_loss, test_acc = evaluate_global(global_model, x_test, y_test)
print(f'Final global model test accuracy: {test_acc * 100:.2f}%')

# 라운드에 따른 글로벌 모델 정확도 변화 그래프
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_rounds + 1), global_model_accuracies, 'r-o', label='Global Model Accuracy')
plt.xlabel('Round')
plt.ylabel('Accuracy (%)')
plt.title('Global Model Accuracy Over Rounds')
plt.legend()
plt.show()

(99, 17472)


ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 1 and 48 for '{{node sequential_2_1/lstm_4_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_2_1/lstm_4_1/strided_slice_1, sequential_2_1/lstm_4_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [32,1], [48,1024].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(32, 1), dtype=float32)
  • states=('tf.Tensor(shape=(32, 256), dtype=float32)', 'tf.Tensor(shape=(32, 256), dtype=float32)')
  • training=False